In [2]:
import requests
import time
import urllib.parse
import hmac
import hashlib
import base64

def get_signature(secret, method, path, params, nonce, timestamp) -> str:
    """生成API签名"""
    text_list = [
        method,
        urllib.parse.quote_plus(path),
        params,
        f"x-api-nonce:{nonce}",
        f"x-api-timestamp:{timestamp}",
        "",
    ]
    text = "\n".join(text_list)
    enc = hmac.new(secret.encode("utf-8"), text.encode("utf-8"), hashlib.sha256).hexdigest()
    return base64.b64encode(enc.encode("utf-8")).decode("utf-8")

def get_token(app_key, app_secret, client_id, client_secret, nonce, timestamp):
    """获取访问令牌"""
    url = "https://api.kingdee.com/jdyconnector/app_management/kingdee_auth_token"
    sig = hmac.new(app_secret.encode("utf-8"), app_key.encode("utf-8"), hashlib.sha256)
    sig_base64 = base64.b64encode(sig.hexdigest().encode("utf-8")).decode("utf-8")
    sig_url = urllib.parse.quote(sig_base64)
    sig_params = urllib.parse.quote(sig_url)
    params = f"app_key={app_key}&app_signature={sig_params}"
    path = urllib.parse.urlparse(url).path
    signature_base64 = get_signature(client_secret, "GET", path, params, nonce, timestamp)
    headers = {
        "Content-Type": "application/json",
        "X-Api-ClientID": str(client_id),
        "X-Api-Auth-Version": "2.0",
        "X-Api-TimeStamp": str(timestamp),
        "X-Api-SignHeaders": "X-Api-TimeStamp,X-Api-Nonce",
        "X-Api-Nonce": str(nonce),
        "X-Api-Signature": signature_base64,
    }
    requset_url = f"{url}?app_key={app_key}&app_signature={sig_url}"
    response = requests.get(requset_url, headers=headers)
    if response.status_code == 200:
        if data := response.json().get("data"):
            return data.get("app-token")
        else:
            raise Exception(response.text)
    else:
        return None

def pur_order_list(url, client_id, client_secret, app_token, nonce, timestamp):
    """获取采购订单列表"""
    path = urllib.parse.urlparse(url).path
    params = urllib.parse.urlparse(url).query
    header = {
        "Content-Type": "application/json",
        "X-Api-ClientID": str(client_id),
        "X-Api-Auth-Version": "2.0",
        "X-Api-TimeStamp": str(timestamp),
        "X-Api-SignHeaders": "X-Api-TimeStamp,X-Api-Nonce",
        "X-Api-Nonce": str(nonce),
        "X-Api-Signature": get_signature(client_secret, "GET", path, params, nonce, timestamp),
        "app-token": app_token,
    }
    response = requests.get(url=url, headers=header)
    return response.json()

def pur_order_detail(url, client_id, client_secret, app_token, nonce, timestamp):
    """获取采购订单详情"""
    path = urllib.parse.urlparse(url).path
    params = urllib.parse.urlparse(url).query
    header = {
        "Content-Type": "application/json",
        "X-Api-ClientID": str(client_id),
        "X-Api-Auth-Version": "2.0",
        "X-Api-TimeStamp": str(timestamp),
        "X-Api-SignHeaders": "X-Api-TimeStamp,X-Api-Nonce",
        "X-Api-Nonce": str(nonce),
        "X-Api-Signature": get_signature(client_secret, "GET", path, params, nonce, timestamp),
        "app-token": app_token,
    }
    response = requests.get(url=url, headers=header)
    return response.json()

def main():
    """主函数 - 获取所有采购订单数据"""
    # API认证信息
    app_key = "4Ad2WyhL"
    app_secret = "0823747c5c3756d2f2e60f09640683b7a3b7173e"
    client_id = "306872"
    client_secret = "884f9f7e0be2cf3b5526c3ec36900aae"
    
    timestamp = str(int(time.time() * 1000))
    nonce = timestamp
    
    # 获取访问令牌
    app_token = get_token(app_key, app_secret, client_id, client_secret, nonce, timestamp)
    if not app_token:
        return {"error": "Failed to get token"}
    
    all_data = []
    page = 1
    page_size = 100
    
    # 分页获取所有采购订单
    while True:
        base_url = "https://api.kingdee.com/jdy/v2/scm/pur_order"
        params = {
            "page": page,
            "page_size": page_size
        }
        sorted_params = dict(sorted(params.items()))
        query_string = "&".join([f"{k}={v}" for k, v in sorted_params.items()])
        list_url = f"{base_url}?{query_string}"
        timestamp = str(int(time.time() * 1000))
        nonce = timestamp
        
        # 获取订单列表
        list_response_text = pur_order_list(list_url, client_id, client_secret, app_token, nonce, timestamp)
        
        if isinstance(list_response_text, dict):
            list_response = list_response_text
        else:
            try:
                list_response = json.loads(list_response_text)
            except json.JSONDecodeError:
                print(f"Failed to parse list response: {list_response_text}")
                break
        
        list_data = list_response.get("data", {})
        rows = list_data.get("rows", [])
        if not rows:
            break
        
        # 获取每个订单的详细信息
        for row in rows:
            order_id = row.get("id")
            if order_id:
                detail_base_url = "https://api.kingdee.com/jdy/v2/scm/pur_order_detail"
                detail_params = {
                    "id": order_id
                }
                sorted_detail_params = dict(sorted(detail_params.items()))
                detail_query_string = "&".join([f"{k}={v}" for k, v in sorted_detail_params.items()])
                detail_url = f"{detail_base_url}?{detail_query_string}"
                timestamp = str(int(time.time() * 1000))
                nonce = timestamp
                
                detail_response_text = pur_order_detail(detail_url, client_id, client_secret, app_token, nonce, timestamp)
                
                if isinstance(detail_response_text, dict):
                    detail_response = detail_response_text
                elif isinstance(detail_response_text, str):
                    try:
                        detail_response = json.loads(detail_response_text)
                    except json.JSONDecodeError:
                        print(f"Failed to parse detail response for order {order_id}: {detail_response_text}")
                        continue
                
                all_data.append(detail_response.get("data", {}))
        
        page += 1
        time.sleep(0.1)  # 防止请求过于频繁
    
    return {"result": all_data}
if __name__ == "__main__":
    result = main()
    print(result)


{'result': [{'id': '2295181852287199232', 'bill_no': 'CGDD-20250902-00001', 'bill_date': '2025-09-02', 'bill_status': 'C', 'create_time': '2025-09-02 17:56:00', 'modify_time': '2025-09-08 11:41:22', 'audit_time': '2025-09-08 11:41:22', 'creator_id': '13525536', 'creator_name': '穆青竹', 'creator_number': 'multicorp_13525536', 'modifier_id': '13525536', 'modifier_name': '穆青竹', 'modifier_number': 'multicorp_13525536', 'auditor_id': '13525536', 'auditor_name': '穆青竹', 'auditor_number': 'multicorp_13525536', 'trans_type': '2', 'remark': '', 'dept_id': '', 'dept_name': '', 'dept_number': '', 'emp_id': '', 'emp_name': '', 'emp_number': '', 'customer_id': '', 'customer_name': '', 'customer_number': '', 'supplier_id': '2289280960027443200', 'supplier_name': '1', 'supplier_number': 'GYS00001', 'contact_phone': '', 'contact_country_id': '', 'contact_country_name': '', 'contact_country_number': '', 'contact_province_id': '', 'contact_province_name': '', 'contact_province_number': '', 'contact_city_id